In [219]:
# 1. Creat demo.xlsx

In [388]:
# import numpy as np
# import pandas as pd
# np.random.seed(1)
# data = pd.DataFrame({
#     'label': np.random.randint(0, 2, 500),
#     'age': np.random.randint(18, 70, 500),
#     'sex': np.random.randint(0, 2, 500),
#     'height': np.random.randint(140, 200, 500),
#     'weight': np.random.randint(40, 100, 500)
# })
# data['bsa'] = round(0.0061 * data['height'] + 0.0128 * data['weight'] - 0.1529, 3)
# data['bmi'] = round(data['weight'] / (data['height']/100)**2, 1)

# data['label'] = data['height'].apply(lambda x : 1 if x in range(155,180) else 0) & \
#                 data['weight'].apply(lambda x : 1 if x in range(45,70) else 0)

# data.to_excel('demo.xlsx', index=False)

In [221]:
# 2. Fast Evaluation

In [389]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

datafile = 'demo.xlsx'
data = pd.read_excel(datafile)
X, y = data.values[:, 1:], data.values[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
result = accuracy_score(y_test, y_predict)
print(result)

1.0


In [417]:
test = pd.DataFrame({
    'age':[23],
    'sex':[1],
    'height':[140],
    'weight':[65]
})
print(model.predict(test))

[0]
